In [29]:
import json
import random
import os
import numpy as np
from PIL import Image
import copy

def get_horizontal_frames(num_image, frames):
    equator_threshold = 0.4
    center_pt = np.zeros(3)
    
    # initialization of dict_list
    selected_frame_dict_list = []
    for i in range(num_image):
        s_f_dict = {}
        theta = 2*np.pi/num_image * i
        base_vector = np.array([np.sin(theta), np.cos(theta)])
        s_f_dict["base_vector"] = base_vector
        s_f_dict["selected_frame_inner_product"] = -1000
        s_f_dict["selected_frame"] = frames[0]
        selected_frame_dict_list.append(s_f_dict)

    # select num_image frames
    for frame in frames:
        transform_matrix = frame["transform_matrix"]
        t_m_array = np.array(transform_matrix) # =~ c2w ?

        # z = t_m_array[2, 3] height??
        if t_m_array[2,3] >= equator_threshold:
            continue
            
        # xy plane vector
        vector = t_m_array[[0, 1], 3] # - center_pt[[2, 1]]
        direction = vector / np.linalg.norm(vector)

        # inner product =~ 1 -> select as a frame
        for i, s_f_dict in enumerate(selected_frame_dict_list):
            # inner_product = np.dot(direction, s_f_dict["base_vector"])
            inner_product = np.dot(direction, s_f_dict["base_vector"])
            if s_f_dict["selected_frame_inner_product"] < inner_product:
                selected_frame_dict_list[i]["selected_frame"] = frame
                selected_frame_dict_list[i]["selected_frame_inner_product"] = inner_product

    selected_frames = [s_f_dict["selected_frame"] for s_f_dict in selected_frame_dict_list]
    return selected_frames

def modify_dict_data_horizontal(num_image, dict_data):
    frames = dict_data["frames"]

    selected_frames = get_horizontal_frames(num_image, frames)

    modified_dict_data = copy.deepcopy(dict_data)
    modified_dict_data["frames"] = selected_frames
    return modified_dict_data
    
def get_dict_data_from_json_path(input_json_path):
    with open(input_json_path, 'r') as f:
        dict_data = json.load(f)
    return dict_data

def save_dict_data(dict_data, output_json_path):
    with open(output_json_path, "w") as f:
        json.dump(dict_data, f)
        #json.dump(dict_data, f, indent=4)
        
def do_io_json_horizontal(in_dir, out_dir):
    in_json_path = f"{in_dir}/train_all.json"
    dict_data = get_dict_data_from_json_path(in_json_path)
    
    for num_image in [3, 10]:
        modified_dict_data = modify_dict_data_horizontal(num_image, dict_data)
        out_json_path = f"{out_dir}/train_all_{num_image}_horizontal.json"
        save_dict_data(modified_dict_data, out_json_path)
    
def make_json_all_scene():
    path = "/home/ccl/Datasets/NeRF/ScanNerf/"
    scene_list = sorted(os.listdir(path))

    for scene in scene_list:
        input_dir_path = f"{path}{scene}/"

        # for test
        new_dir = f"./train_horizontal"
        if not os.path.exists(new_dir):
            os.mkdir(new_dir)

        new_dir += f"/{scene}"
        if not os.path.exists(new_dir):
            os.mkdir(new_dir)

        # for actual purpose
        # new_dir = f"{path}{scene}/" 
        
        do_io_json_horizontal(input_dir, new_dir)
        
# image
ckpt_path = "/home/ccl/Datasets/NeRF/ScanNerf/"
scene_list = sorted(os.listdir(ckpt_path))

def copy_image_of_json(in_json_path, in_image_dir, out_dir, scene):
    with open(in_json_path, 'r') as f:
        dict_data = json.load(f)
    frames = dict_data["frames"]
    
    if not os.path.exists(out_dir):
        os.mkdir(out_dir)
    
    imgs = []
    for i, frame in enumerate(frames):
        file_path = frame["file_path"]
        img_path = f"{in_image_dir}/{file_path}.png"
        img = Image.open(img_path)
        img.save(f"{out_dir}/{i}_{os.path.basename(file_path)}.png")
        imgs.append(img)

    imgs[0].save(f"{out_dir}/{scene}.gif",save_all=True, append_images=imgs[1:],optimize=True, duration=200, loop=0)

    
def save_image_jsons(num_train_image):
    for scene in scene_list:
        in_json_dir = f"./train_horizontal/{scene}/"
        in_json_path = f"{in_json_dir}/train_all_{num_train_image}_horizontal.json"
        in_image_dir = f"/home/ccl/Datasets/NeRF/ScanNerf/{scene}"
        out_dir = f"{in_json_dir}/{num_train_image}"
        copy_image_of_json(in_json_path, in_image_dir, out_dir, scene)

        
# for test!!! only 1 scene(airplane1)

def make_json_img_test(scene):
    train_data_ckpt_dir = "/home/ccl/Datasets/NeRF/ScanNerf"
    in_dir = f"{train_data_ckpt_dir}/{scene}/"
    in_dir = "."
    out_dir = f"./test/{scene}"
    if not os.path.exists(out_dir):
        os.mkdir(out_dir)
    do_io_json_horizontal(in_dir, out_dir)

    for num_train_image in [3, 10]:
        in_json_dir = f"./test/{scene}/"
        in_json_path = f"{in_json_dir}/train_all_{num_train_image}_horizontal.json"
        in_img_dir = f"/home/ccl/Datasets/NeRF/ScanNerf/{scene}"
        out_img_dir = f"{out_dir}/{num_train_image}"

        copy_image_of_json(in_json_path, in_img_dir, out_img_dir, scene)
        
make_json_img_test("airplane1")

In [2]:
num_image = 10

for i in range(num_image):
    s_f_dict = {}
    theta = 2*np.pi/num_image * i
    base_vector = np.array([np.sin(theta), np.cos(theta)])
    s_f_dict["base_vector"] = base_vector
    print(theta, np.degrees(theta))


0.0 0.0
0.6283185307179586 36.0
1.2566370614359172 72.0
1.8849555921538759 108.0
2.5132741228718345 144.0
3.141592653589793 180.0
3.7699111843077517 216.0
4.39822971502571 252.0
5.026548245743669 288.0
5.654866776461628 324.0
